In [2]:
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import datetime
import requests
import pandas as pd

In [3]:
def get_page_source(url):
    agent = {"User-Agent":"Mozilla/5.0"} # User Agent defined to avoid robot detection
    source=requests.get(url, headers=agent)
    return source.text, source.status_code


def get_clean_text(content):
    raw_html=str(content)
    text=(BeautifulSoup(raw_html, "lxml").text)
    return text

def get_soup(content):
    soup=BeautifulSoup(content, "lxml")
    return soup

In [4]:
def get_data_from_table(url = "https://www.globalbrandsmagazine.com/list-of-banks-by-country"):
    
    text, status = get_page_source(url)
    if status!= 200: # HTTP request Error encountered
        return "Error"
    soup = get_soup(text)
    
    divs= soup.findAll('div', attrs = {'class': "theiaPostSlider_preloadedSlide"})
    data=list()
    if not len(divs): # No Table
        return data
    else:
        for div in divs: # If more than one table encountered
            table = div.find('table')
            rows = table.findAll("td")
            for row in rows:
                if(len(row.findAll('strong'))): # Added specifically for parsing Country header
                    country = row.text.strip()
                    
                else:
                    raw = row.text.strip()
                    # Change last line according to the information 
                    data.append({'Country' : country, 'Bank' : raw})
        return data

In [5]:
data = get_data_from_table()
df = pd.DataFrame(data)

In [6]:
df.to_csv("list_of_banks.csv")
df

,Country,Bank
0,Afghanistan,Afghanistan International Bank (Local Bank)
1,Afghanistan,Afghanistan Commercial Bank (Local Bank)
2,Afghanistan,Ghazanfar Bank (Local Bank)
3,Afghanistan,Maiwand Bank (Local Bank)
4,Afghanistan,Afghan United Bank (Local Bank)
...,...,...
4426,Zambia,Investrust Bank (Commercial Bank)
4427,Zambia,Stanbic Bank (Commercial Bank)
4428,Zambia,Standard Chartered Bank (Zambia) (Commercial B...
4429,Zambia,United Bank for Africa (Commercial Bank)
